In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential

%load_ext tensorboard
import datetime

from tensorflow.keras.utils import to_categorical

filename = "../../Datasets/CHD/heart.csv"

df = pd.read_csv(filename)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
initializer0 = keras.initializers.RandomUniform(minval = 0, maxval =0.005)
initializer1 = keras.initializers.RandomUniform(minval = 0, maxval =0.005)
initializer2 = keras.initializers.RandomUniform(minval = -2, maxval =1)

class Diffact(keras.layers.Layer):
    def __init__(self):
        super(Diffact, self).__init__()
        self.k0 = self.add_weight(name='k0', shape = (), initializer=initializer0, trainable=True)
        self.k1 = self.add_weight(name='k1', shape = (), initializer=initializer1, trainable=True)
        self.k2 = self.add_weight(name='k2', shape = (), initializer=initializer2, trainable=True)
        
    def call(self, inputs):
        return self.k0 + tf.multiply(inputs, self.k1) + tf.multiply(tf.multiply(inputs,inputs), self.k2)

In [3]:
#Change the column names for better understanding
df.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

In [4]:
#Finding the shape of the dataframe
df.shape

(303, 14)

In [5]:
# Describing the Dataframe
df.describe()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
#Finding information of dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      303 non-null    int64  
 1   sex                      303 non-null    int64  
 2   chest_pain_type          303 non-null    int64  
 3   resting_blood_pressure   303 non-null    int64  
 4   cholesterol              303 non-null    int64  
 5   fasting_blood_sugar      303 non-null    int64  
 6   rest_ecg                 303 non-null    int64  
 7   max_heart_rate_achieved  303 non-null    int64  
 8   exercise_induced_angina  303 non-null    int64  
 9   st_depression            303 non-null    float64
 10  st_slope                 303 non-null    int64  
 11  num_major_vessels        303 non-null    int64  
 12  thalassemia              303 non-null    int64  
 13  target                   303 non-null    int64  
dtypes: float64(1), int64(13)
m

In [7]:
#Finding the missing values. In this dataframe there is no missing value.
df.isnull().sum()

age                        0
sex                        0
chest_pain_type            0
resting_blood_pressure     0
cholesterol                0
fasting_blood_sugar        0
rest_ecg                   0
max_heart_rate_achieved    0
exercise_induced_angina    0
st_depression              0
st_slope                   0
num_major_vessels          0
thalassemia                0
target                     0
dtype: int64

In [8]:
# Select the columns to use for prediction in the neural network
X= df.drop('target',axis=1)
Y=df['target']
print (X.shape, Y.shape, df.columns)
Y=np.array(Y)
Y=to_categorical(Y)

(303, 13) (303,) Index(['age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
       'cholesterol', 'fasting_blood_sugar', 'rest_ecg',
       'max_heart_rate_achieved', 'exercise_induced_angina', 'st_depression',
       'st_slope', 'num_major_vessels', 'thalassemia', 'target'],
      dtype='object')


In [9]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
# split data into train, test
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=39, shuffle=True)
#kipping y since value already 1 or 0
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)

# normalize data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)
X_test_scaled = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test_scaled)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)
print (df.columns)

(212, 13) (212, 2)
(91, 13) (91, 2)
Index(['age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
       'cholesterol', 'fasting_blood_sugar', 'rest_ecg',
       'max_heart_rate_achieved', 'exercise_induced_angina', 'st_depression',
       'st_slope', 'num_major_vessels', 'thalassemia', 'target'],
      dtype='object')


In [10]:
# create model with fully connected layers with dropout regulation
model = Sequential()
model.add(layers.Dense(25, input_dim=13))
diffact = Diffact()
model.add(diffact)
model.add(layers.Dropout(0.1))
model.add(layers.Dense(2, activation = 'softmax'))
#model.add(diffact)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                350       
_________________________________________________________________
diffact (Diffact)            (None, 25)                3         
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 52        
Total params: 405
Trainable params: 405
Non-trainable params: 0
_________________________________________________________________


In [11]:
batch_size = 5
epochs = 100

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=0.001), metrics=["accuracy"])

log_dir = "heartlogs/smallk2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.33 ,callbacks=[tensorboard_callback], verbose=1)


Epoch 1/100
 1/29 [>.............................] - ETA: 0s - loss: 0.9756 - accuracy: 0.2000WARNING:tensorflow:From C:\Users\shamb\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
29/29 [==============================] - 0s 14ms/step - loss: 0.6555 - accuracy: 0.6197 - val_loss: 0.5408 - val_accuracy: 0.7571
Epoch 2/100
29/29 [==============================] - 0s 3ms/step - loss: 0.5425 - accuracy: 0.7254 - val_loss: 0.5027 - val_accuracy: 0.7714
Epoch 3/100
29/29 [==============================] - 0s 3ms/step - loss: 0.5357 - accuracy: 0.7394 - val_loss: 0.4705 - val_accuracy: 0.8143
Epoch 4/100
29/29 [==============================] - 0s 3ms/step - loss: 0.5142 - accuracy: 0.7606 - val_loss: 0.4542 - val_accuracy: 0.8000
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss

29/29 [==============================] - 0s 2ms/step - loss: 0.2714 - accuracy: 0.9014 - val_loss: 0.4825 - val_accuracy: 0.7857
Epoch 54/100
29/29 [==============================] - 0s 3ms/step - loss: 0.2863 - accuracy: 0.8944 - val_loss: 0.4804 - val_accuracy: 0.7857
Epoch 55/100
29/29 [==============================] - 0s 3ms/step - loss: 0.2731 - accuracy: 0.9014 - val_loss: 0.4729 - val_accuracy: 0.7714
Epoch 56/100
29/29 [==============================] - 0s 2ms/step - loss: 0.2732 - accuracy: 0.9014 - val_loss: 0.4663 - val_accuracy: 0.7714
Epoch 57/100
29/29 [==============================] - 0s 2ms/step - loss: 0.3051 - accuracy: 0.8873 - val_loss: 0.4806 - val_accuracy: 0.7714
Epoch 58/100
29/29 [==============================] - 0s 2ms/step - loss: 0.2905 - accuracy: 0.9014 - val_loss: 0.4845 - val_accuracy: 0.7571
Epoch 59/100
29/29 [==============================] - 0s 2ms/step - loss: 0.2815 - accuracy: 0.9085 - val_loss: 0.4827 - val_accuracy: 0.7571
Epoch 60/100
29/29 

In [12]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.5003658533096313
Test accuracy: 0.8131868243217468


In [13]:
print("AF coefficients (weights) {}".format(diffact.get_weights()))

AF coefficients (weights) [0.17747906, -0.094401166, -2.2689955]


In [15]:
%tensorboard --logdir heartlogs/smallk2 --port=8042

Reusing TensorBoard on port 8042 (pid 17760), started 0:00:52 ago. (Use '!kill 17760' to kill it.)